# VGG 논문
https://arxiv.org/pdf/1409.1556

In [1]:
import tensorflow as tf

* 3차원 이미지를 대상으로 CNN구조를 만들 때!!!!   
    --> 집중의 대상은 2차원 사이즈 중심!!!  
    ( 뒤에 있는 채널에 대한 것들은 생략!!!!!--> 니가 알아서 연결)  
    --> 필터의 수와 연동이 되니까 코드가 알아서...

In [ ]:
# 1) Conv2D 레이어에 대한 세팅!!(채널은 신경 안 쓰겠다!!!)
# - kernel_size : 필터에 대한 2차원 사이즈(어느 사이즈로 스캔할지)
#                 3*3, 5*5, 7*7 etc 특징들을 추출..,
#                 ==> Output인 Feature Map의 사이즈에 연동!!!
# - filters : 몇 개의 필터를 사용할지.....
#             <==> Feature Map을 몇 장 만들어 낼지!!!!!!!
#                  output인 Feature Map의 Channel의 수!!!
# - stride  : 가로 step, 세로 step
#             얼마나 자세하게 스캐닝을 할지/말지..
#             ==> Feature Map의 Size에 연관!!!!
# - padding : 테두리에 대한 처리( 사이즈 보전을 할지 말지..)
#             valid : 그냥 테두리 없이 스캐닝하자
#             same  : 니가 내가 지정한 kernel_size에 맞춰서
#                     입력과 동일한 Feature Map이 나오도록
#                      테두리 처리를 해주라!!!
# +++ ActivationFunction : 초기 버전들은 AF을 사용X
#                          VGG 모델 전후로 AF을 사용을 함...

In [2]:
conv1 = tf.keras.layers.Conv2D(
    # 우리가 신경쓸 부분은 오로지 2D!!!
    # 나머지 : channel은 코드가 알아서!!!!
    kernel_size = (5,5),
    filters = 4,
    strides = (2,2),
    padding = "same"
    # ++ 최근에는 Activation Function
    # activation="relu"
)

In [ ]:
# Pooling Layer
# --> Feature Map의 결과들을 대표화를 하는 과정!!!!!
#     ( 일반적으로 사이즈들을 줄이는 것을 선호)
# - pool_size : 처리가 되는 단위 : 대표화의 단위( 2,2)
# - strides : 옵션도 같이 활용....

pool1 = tf.keras.layers.MaxPool2D(
    pool_size = (2,2),
    strides = (2,2)
)

In [ ]:
# + 연결이 많아서....
# --> 중간에 적당학게 연결을 짜르는 Dropout
# 참고) 최종적인 FeatureMap을 분류DNN구조와 연결을 할 때...
#       FullConnection을 주로 활용을 함...

#### 데이터 불러오기

In [3]:
fashion_mnist = tf.keras.datasets.fashion_mnist
(train_X, train_y),(test_X, test_y) = fashion_mnist.load_data()


print(train_X.shape)
print(train_y.shape)
print(test_X.shape)
print(test_y.shape)

4422102/4422102 [==============================] - 0s 0us/step
(60000, 28, 28)
(60000,)
(10000, 28, 28)
(10000,)


In [4]:
# 전처리 : 이미지 전처리 --> 1/255.0으로 정규화
train_X = train_X/255.0
test_X = test_X /255.0

In [ ]:
# ===> 간단하게 하기 위해서 흑백 사진을 사용을 하였음..
# 보통은 컬러이미지를 가지고 수행을 함!!!!
# + 컬러 자체로 할지 // 흑백으로 채널을 줄여서 할지..결정!!!!
# ( 컬러의 특징이 중요하지 않을 때... )

# ==> 전통적인 이미지 처리 : openCV(c/c++--> 모든 언어)
#     colab의 opencv는 일반적인 opencv와 채널의 순서가 다름..
#     푸르딩딩으로 나옴;;;RGB--> BGR

In [ ]:
# 1. 입력에 대한 데이터 처리!!!!!!!
# ===> 1장에 대해서 처리!!!!!!
#       n장에 대해서 일괄 처리는 코드가 TF가 알아서 함.....
# ==> 1장에 대해서 잘 코드화!!!!!!

In [5]:
print(train_X.shape)
print(train_X[0].shape)


# 모델의 구조도에 맞춰서 데이터셋을 변형!!!!!
# 3d ---> 4d (데이터 수, 가로, 세로, 채널)
# 현재 데이터 셋 : 60000, 28 X 28 X 1 [3D]
# 원하는 입력 데이터 셋 : 60000, 28, 28, 1 [4D]
# 이런 식으로 입력 데이터셋을 변경해야 신경 쓸 일이 없음

train_X = train_X.reshape(-1 ,28,28,1)
test_X  = test_X.reshape(-1, 28,28,1)

print(train_X.shape)
print(train_X[0].shape)


(60000, 28, 28)
(28, 28)
(60000, 28, 28, 1)
(28, 28, 1)


In [6]:
#### 가장 간단한 CNN구조 모델!!!!
model = tf.keras.Sequential([

    # Step1) 입력 + 이미지 특징 추출
    # 입력 : 1장 데이터를 기준 --> 28,28,1// (60000,28,28,1)
    tf.keras.layers.Conv2D( input_shape=(28,28,1),
                            kernel_size=(3,3),
                            filters = 16
                            ),

    # 2번 conv 레이어
    tf.keras.layers.Conv2D(kernel_size=(3,3),
                           filters=32
                           ),

    # 3번 conv 레이터
    tf.keras.layers.Conv2D(kernel_size=(3,3),
                           filters=64),



    # ==== (이미지가 가진 특징을 잘 뽑아냈다고 가정) ==== #


    # Step2) 분류를 위한 DNN구조!!!!!
    # 차원을 변경
    tf.keras.layers.Flatten(),

    # ++ 일반적 분류를 위한 레이어 추가
    #    노드 수, AF, 몇 개 레이어를 쌓을지 HP
    tf.keras.layers.Dense( units = 128, activation="relu"),

    # Step3) 출력용
    tf.keras.layers.Dense( units = 10, activation="softmax")
])
model

In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_1 (Conv2D)           (None, 26, 26, 16)        160       
                                                                 
 conv2d_2 (Conv2D)           (None, 24, 24, 32)        4640      
                                                                 
 conv2d_3 (Conv2D)           (None, 22, 22, 64)        18496     
                                                                 
 flatten (Flatten)           (None, 30976)             0         
                                                                 
 dense (Dense)               (None, 128)               3965056   
                                                                 
 dense_1 (Dense)             (None, 10)                1290      
                                                                 
Total params: 3989642 (15.22 MB)
Trainable params: 39896

In [ ]:
# 기본적인 cnn구조로 설계한 네트워크의 학습!
# 참고) 정답을 분류 -> 원핫인코딩 FM : 너무 귀찮음!
# loss 함수를 지정하는 과정에서 앞에 sparse_ ~~
# 굳이 정답을 원핫 인코딩을 안 하고, 라벨 인코딩으로 넣어도 됨!

# 참고) 여기서는 정답지(라벨인코딩된 거 그대로 사용)


In [ ]:
train_y

array([9, 0, 0, ..., 3, 0, 5], dtype=uint8)

#### 모델 학습 세팅

In [8]:
model.compile(
    # 이런 loss는 정답을 무조건 one_hot 인코딩
    # loss = "categorical_crossentropy" 에서
    # sparse 를 붙이면 라벨 인코딩 가능
    loss = "sparse_categorical_crossentropy",
    optimizer = tf.keras.optimizers.Adam(),
    metrics = ["accuracy"]

)

#### 모델 학습

In [9]:
# 실제 학습을 진행
history = model.fit(
    train_X, train_y,
    epochs = 20,
    validation_split = 0.25,
    batch_size = 128
)

Epoch 1/20
352/352 [==============================] - 10s 12ms/step - loss: 0.5007 - accuracy: 0.8232 - val_loss: 0.3829 - val_accuracy: 0.8598
Epoch 2/20
352/352 [==============================] - 3s 9ms/step - loss: 0.3423 - accuracy: 0.8755 - val_loss: 0.3678 - val_accuracy: 0.8657
Epoch 3/20
352/352 [==============================] - 3s 10ms/step - loss: 0.2839 - accuracy: 0.8960 - val_loss: 0.3616 - val_accuracy: 0.8737
Epoch 4/20
352/352 [==============================] - 3s 10ms/step - loss: 0.2400 - accuracy: 0.9120 - val_loss: 0.3787 - val_accuracy: 0.8675
Epoch 5/20
352/352 [==============================] - 3s 9ms/step - loss: 0.2047 - accuracy: 0.9243 - val_loss: 0.3860 - val_accuracy: 0.8758
Epoch 6/20
352/352 [==============================] - 3s 10ms/step - loss: 0.1721 - accuracy: 0.9377 - val_loss: 0.4351 - val_accuracy: 0.8725
Epoch 7/20
352/352 [==============================] - 4s 10ms/step - loss: 0.1500 - accuracy: 0.9458 - val_loss: 0.4824 - val_accuracy: 0.8695


In [10]:
# conv : stride : 샘플링에 대한 빈도
# pooling : 대표화 (Feature Map 에 대한 대표화)
# + 분류에서 DNN dropout 활용해서 연결을 끊어내기
# -> overfit 줄이고자 함

# 초기 CNN : conv + pool + conv + pool
# VGG 논문 이후 : conv + conv + conv + pool etc



In [11]:
# 시도1) 초기 버전으로 시도...
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D( input_shape=(28,28,1),kernel_size=(3,3),filters = 16),
    tf.keras.layers.MaxPooling2D( pool_size=(2,2), strides=(2,2)),
    tf.keras.layers.Conv2D(kernel_size=(3,3), filters=32),
    tf.keras.layers.MaxPooling2D( pool_size=(2,2), strides=(2,2)),
    tf.keras.layers.Conv2D(kernel_size=(3,3), filters=64),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout( rate= 0.3),
    tf.keras.layers.Dense( units = 128, activation="relu"),
    tf.keras.layers.Dropout( rate= 0.3),

    tf.keras.layers.Dense( units = 10, activation="softmax")
])
model

In [12]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 26, 26, 16)        160       
                                                                 
 max_pooling2d (MaxPooling2  (None, 13, 13, 16)        0         
 D)                                                              
                                                                 
 conv2d_5 (Conv2D)           (None, 11, 11, 32)        4640      
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 5, 5, 32)          0         
 g2D)                                                            
                                                                 
 conv2d_6 (Conv2D)           (None, 3, 3, 64)          18496     
                                                                 
 flatten_1 (Flatten)         (None, 576)              

In [ ]:
# 동일한 구조인대 대표화 & 중간에 짜르기
# 400만개 파라미터  -> 10만개 정도로 줄이는 것
# 내가 어떻게 설계하는지에 따라서 엄청나게 다양성이 존재

#### 모델 셋팅 및 학습

In [13]:
model.compile(
    loss = "sparse_categorical_crossentropy",
    optimizer = tf.keras.optimizers.Adam(),
    metrics = ["accuracy"]
)

In [14]:
history = model.fit(
    train_X, train_y,
    epochs = 20,
    validation_split =0.25,
    batch_size= 128
)

Epoch 1/20
352/352 [==============================] - 5s 6ms/step - loss: 0.6993 - accuracy: 0.7425 - val_loss: 0.4782 - val_accuracy: 0.8186
Epoch 2/20
352/352 [==============================] - 2s 5ms/step - loss: 0.4629 - accuracy: 0.8330 - val_loss: 0.3810 - val_accuracy: 0.8600
Epoch 3/20
352/352 [==============================] - 2s 5ms/step - loss: 0.3995 - accuracy: 0.8532 - val_loss: 0.3596 - val_accuracy: 0.8695
Epoch 4/20
352/352 [==============================] - 2s 7ms/step - loss: 0.3664 - accuracy: 0.8693 - val_loss: 0.3454 - val_accuracy: 0.8735
Epoch 5/20
352/352 [==============================] - 2s 6ms/step - loss: 0.3457 - accuracy: 0.8748 - val_loss: 0.3131 - val_accuracy: 0.8858
Epoch 6/20
352/352 [==============================] - 2s 5ms/step - loss: 0.3302 - accuracy: 0.8807 - val_loss: 0.3071 - val_accuracy: 0.8867
Epoch 7/20
352/352 [==============================] - 2s 5ms/step - loss: 0.3170 - accuracy: 0.8837 - val_loss: 0.3032 - val_accuracy: 0.8883
Epoch 

In [15]:
#### 모델링에서 중요한 기본적인 부분은 Bias-Variance
#     ==> UnderFit ~~~ 적당한fit ~~~~ overFit
# 기존의 ML ) HPT을 중심으로 해당하는 모델의 학습을
#            max_depth etc
# Deep Learning ) 모델의 구조를 통해서 조절
#             : 연결성을 중심으로 처리를 함


# ---> 모델을 경량화를 했더니 학습이 더 잘된다!!!
# 더 학습을 해도 될 것 같다
# 지금까지는 OverFit은 아닌거 같으니, 더 해보고 싶다
# callback 을 사용해서 모니터링을 해야함
# + 중간에 때려칠지 & + 중간에 모델 저장

#### 모델 저장
    * ckpt

In [16]:
import os

In [18]:
# --> epochs를 일단 크게 세팅을 하고,,,
#     early stop을 통해서 더이상 성능 향상이 없으면 stop
# --> 중간 중간 모델의 weights를 저장(모델 저장)
cp_path = "training/cp-{epoch:04d}.ckpt"
cp_dir = os.path.dirname(cp_path) # training
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    cp_path,
    verbose = 1,
    save_weights_only = True
)

es_callback = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    patience = 10 # 참을 횟수...갱신이 안 되는 횟수..
)

history = model.fit( train_X, train_y,
                    epochs = 20, #아까 20번 돌아서 마지막 부분에서부터 이어서 시작함.엄밀히 말하면 220번째임
                     validation_split = 0.25,
                     batch_size = 128,
                     callbacks = [cp_callback, es_callback])

Epoch 1/20
345/352 [============================>.] - ETA: 0s - loss: 0.2208 - accuracy: 0.9175
Epoch 1: saving model to training/cp-0001.ckpt
352/352 [==============================] - 2s 5ms/step - loss: 0.2204 - accuracy: 0.9176 - val_loss: 0.2727 - val_accuracy: 0.9032
Epoch 2/20
346/352 [============================>.] - ETA: 0s - loss: 0.2165 - accuracy: 0.9199
Epoch 2: saving model to training/cp-0002.ckpt
352/352 [==============================] - 2s 5ms/step - loss: 0.2165 - accuracy: 0.9199 - val_loss: 0.2834 - val_accuracy: 0.8955
Epoch 3/20
350/352 [============================>.] - ETA: 0s - loss: 0.2125 - accuracy: 0.9201
Epoch 3: saving model to training/cp-0003.ckpt
352/352 [==============================] - 2s 5ms/step - loss: 0.2126 - accuracy: 0.9201 - val_loss: 0.2769 - val_accuracy: 0.9027
Epoch 4/20
343/352 [============================>.] - ETA: 0s - loss: 0.2119 - accuracy: 0.9194
Epoch 4: saving model to training/cp-0004.ckpt
352/352 [==========================

In [ ]:
# 참고) 위에서는 처음 20에포크 돌린 이후에 epoch세팅만큼 돌 거라
#       시작점이 높이 시작을 함!!!! 0.9X

In [19]:
# 앞에서 학습한 결과들을 바탕으로 제일 좋을 것 같은.
# 모델의 weights를 가지고 오겠습니다!!!
# ==> 상황마다 다르기에,,,스스로 결정하셔야 함!!!!
# 주의사항) 폴더에 보이는 파일명이 아니라 내가 저장한 양식대로
load_cp_weights = "training/cp-0011.ckpt"

# 모델의 골조 : model 변수.....weights 담겨는 있음..
# --> 구조는 동일하니, 안에 인테리어만 싹 가는것!!!!
# 참고) 다른 사람이 사용할 때에는 모델 구조도 전달을
model.load_weights(load_cp_weights )

# 실제 평가!!!!!!!!!
model.evaluate( test_X, test_y)
# --> 정답지를 그냥 활용할 수 있는 이유는
#     compile에서 loss에  sparse로 설정을 해서임!!!!!

313/313 [==============================] - 1s 3ms/step - loss: 0.2921 - accuracy: 0.8959


[0.29213982820510864, 0.8959000110626221]

In [ ]:
# 대략 구조는 파악을 하고,,,
# 완전히 최적을 해야한다면, HPT을 한다면,

# 구조적인 부분에 대한 여러 실험들을 해야함
# + 필터의 수, 필터의 사이즈, 레이어의 수 etc

# ===> optuna로 실험을 할 수 있음!!!!!!!!!!!!!
# 단, 시간이 엄청 걸리는 부분

In [ ]:
# 개선방향 (논문, 리포트 분석 : 논문대략적이 것들을 파악)
    # -> 잘 되는 모델들을 조사해서 가져다가 사용하자
# 데이터를 보강하자
    # -> 잘 되는 모델을 가져다가 사용하자( 조금 튜닝 )